In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")


In [2]:
from penzai import pz
import json

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import random
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector


In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [20]:
task = "person_profession"
layer = 13

In [7]:
from micrlhf.utils.load_sae import sae_encode, get_nev_it_sae_suite

sae = get_nev_it_sae_suite(layer)

In [14]:

import jax
from penzai import pz


from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)


In [15]:

def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs


In [16]:
# %%
sep = 3978
pad = 0


In [17]:
# n_few_shots, batch_size, max_seq_len = 64, 64, 512
n_few_shots, batch_size, max_seq_len = 20, 16, 256

prompt = "Follow the pattern:\n{}"

In [18]:
from sprint.task_vector_utils import ICLRunner, logprob_loss, get_tv, make_act_adder

from micrlhf.utils.load_sae import sae_encode, sae_encode_gated

from safetensors import safe_open
from sprint.task_vector_utils import FeatureSearch


In [23]:
layer = 14

pairs = list(tasks[task].items())

seed = 10

n_shot = n_few_shots - 1

runner = ICLRunner(task, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt)


tokenized = runner.get_tokens([
    x[:n_shot] for x in runner.train_pairs
], tokenizer)

inputs = tokenized_to_inputs(**tokenized)
train_tokens = tokenized["input_ids"]

_, resids = get_resids_call(inputs)

resids = resids[layer].value.unwrap(
    "batch", "seq", "embedding"
)

tv = get_tv(resids, train_tokens, shift = 0, sep=sep)

tokenized = runner.get_tokens(runner.eval_pairs, tokenizer)
inputs = tokenized_to_inputs(**tokenized)
tokens = tokenized["input_ids"]

add_act = make_act_adder(llama, tv.astype('bfloat16'), tokens, layer, length=1, shift= 0, sep=sep)

logits = add_act(inputs)

loss = logprob_loss(
    logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2, sep=sep, pad_token=0
)

print(
    f"TV: {task}, L: {layer}, Loss: {loss}"  
)

TV: person_profession, L: 14, Loss: 4.09375
